In [ ]:
get_ipython().getoutput("pip install -q segmentation-models-pytorch albumentations")


import os
import cv2
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# --- STEP 1: LIBRARIES ---
try:
    import segmentation_models_pytorch as smp
except:
    os.system('pip install -q segmentation_models_pytorch albumentations')
    import segmentation_models_pytorch as smp

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torch.amp import GradScaler, autocast

# --- STEP 2: CONFIGURATION ---
CFG = {
    "img_size": 512,
    "batch_size": 8,
    "epochs": 25,           # Overfitting se bachne ke liye reduced
    "patience": 7,          # Early stopping: itne epochs tak improve nahi hua toh stop
    "lr": 2e-4,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "train_img": "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images",
    "train_mask": "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks",
    "test_dir": "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/test_images_padded",
    "model_path": "best_terrain_model.pth",
    "csv_path": "submission.csv"
}

# --- STEP 3: AUGMENTATIONS ---
train_transform = A.Compose([
    A.Resize(CFG["img_size"], CFG["img_size"]),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Normalize(),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG["img_size"], CFG["img_size"]),
    A.Normalize(),
    ToTensorV2()
])

# --- STEP 4: DATASET ---
class OffroadDataset(Dataset):
    def __init__(self, images, img_dir, mask_dir, transform=None):
        self.images = images
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform

    def __len__(self): return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = cv2.cvtColor(cv2.imread(os.path.join(self.img_dir, img_name)), cv2.COLOR_BGR2RGB)
        mask_name = os.path.splitext(img_name)[0] + ".png"
        mask = cv2.imread(os.path.join(self.mask_dir, mask_name), cv2.IMREAD_GRAYSCALE)
        mask = (mask > 0).astype(np.float32)
        if self.transform:
            aug = self.transform(image=img, mask=mask)
            img, mask = aug['image'], aug['mask']
        return img, mask.unsqueeze(0)

# --- STEP 5: HELPERS ---
def get_iou(preds, labels):
    preds = (torch.sigmoid(preds) > 0.5).float()
    intersection = (preds * labels).sum()
    union = (preds + labels).sum() - intersection
    return (intersection + 1e-7) / (union + 1e-7)

def rle_encode(mask01):
    pixels = mask01.flatten(order="F") 
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

# --- STEP 6: MAIN ---
def main():
    # Split Data to monitor overfitting
    all_images = sorted([f for f in os.listdir(CFG["train_img"]) if f.lower().endswith(('.png', '.jpg'))])
    train_imgs, val_imgs = train_test_split(all_images, test_size=0.15, random_state=42)

    train_ds = OffroadDataset(train_imgs, CFG["train_img"], CFG["train_mask"], train_transform)
    val_ds = OffroadDataset(val_imgs, CFG["train_img"], CFG["train_mask"], val_transform)
    
    train_loader = DataLoader(train_ds, batch_size=CFG["batch_size"], shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=CFG["batch_size"], shuffle=False, num_workers=2)

    model = smp.UnetPlusPlus("efficientnet-b5", encoder_weights="imagenet", in_channels=3, classes=1).to(CFG["device"])
    optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["lr"])
    criterion = smp.losses.DiceLoss(mode='binary')
    scaler = GradScaler()
    
    best_val_iou = 0
    counter = 0 # For Early Stopping

    for epoch in range(1, CFG["epochs"] + 1):
        # Training
        model.train()
        t_iou = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{CFG['epochs']} [Train]")
        for imgs, masks in pbar:
            imgs, masks = imgs.to(CFG["device"]), masks.to(CFG["device"])
            optimizer.zero_grad()
            with autocast(device_type='cuda'):
                out = model(imgs)
                loss = criterion(out, masks)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            t_iou += get_iou(out, masks).item()
        
        # Validation
        model.eval()
        v_iou = 0
        with torch.no_grad():
            for imgs, masks in val_loader:
                imgs, masks = imgs.to(CFG["device"]), masks.to(CFG["device"])
                out = model(imgs)
                v_iou += get_iou(out, masks).item()
        
        avg_train_iou = t_iou / len(train_loader)
        avg_val_iou = v_iou / len(val_loader)
        
        print(f"Epoch {epoch} -> Train IoU: {avg_train_iou:.4f} | Val IoU: {avg_val_iou:.4f}")

        # Early Stopping & Best Model Check
        if avg_val_iou > best_val_iou:
            best_val_iou = avg_val_iou
            torch.save({'model_state_dict': model.state_dict()}, CFG["model_path"])
            print(f"⭐ Best Val IoU Updated: {best_val_iou:.5f}")
            counter = 0
        else:
            counter += 1
            if counter >= CFG["patience"]:
                print(f"🛑 Early stopping! No improvement for {CFG['patience']} epochs.")
                break
        
        gc.collect()
        torch.cuda.empty_cache()

    # Inference
    print("\n🎬 Starting Submission Generation...")
    checkpoint = torch.load(CFG["model_path"])
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    test_imgs = sorted([f for f in os.listdir(CFG["test_dir"]) if f.endswith(('.png', '.jpg'))])
    rows = []
    with torch.no_grad():
        for name in tqdm(test_imgs, desc="Predicting"):
            img = cv2.cvtColor(cv2.imread(os.path.join(CFG["test_dir"], name)), cv2.COLOR_BGR2RGB)
            input_tensor = val_transform(image=img)["image"].unsqueeze(0).to(CFG["device"])
            out = torch.sigmoid(model(input_tensor)).cpu().numpy()[0, 0]
            mask01 = (cv2.resize(out, (960, 540), interpolation=cv2.INTER_NEAREST) > 0.5).astype(np.uint8)
            rows.append({"image_id": os.path.splitext(name)[0], "encoded_pixels": rle_encode(mask01)})

    pd.DataFrame(rows).to_csv(CFG["csv_path"], index=False)
    print("🏁 Done!")

if __name__ == "__main__":
    main()


import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
from albumentations import Resize, Normalize, Compose
from albumentations.pytorch import ToTensorV2

# --- CONFIG ---
MODEL_PATH = "best_terrain_model.pth" # Aapka saved model file
IMG_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images"
MASK_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def visualize_preds(num_samples=3):
    # 1. Load Model
    # Note: Encoder name wahi rakhein jo training mein use kiya tha (e.g., efficientnet-b5)
    model = smp.UnetPlusPlus("efficientnet-b5", encoder_weights=None, in_channels=3, classes=1).to(DEVICE)
    
    if not os.path.exists(MODEL_PATH):
        print(f"❌ Error: {MODEL_PATH} nahi mili!")
        return
        
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # 2. Preprocessing
    transform = Compose([Resize(512, 512), Normalize(), ToTensorV2()])

    # 3. Get Random Images
    all_imgs = os.listdir(IMG_DIR)
    indices = np.random.choice(len(all_imgs), num_samples, replace=False)

    plt.figure(figsize=(15, 5 * num_samples))

    for i, idx in enumerate(indices):
        img_name = all_imgs[idx]
        img_path = os.path.join(IMG_DIR, img_name)
        mask_path = os.path.join(MASK_DIR, os.path.splitext(img_name)[0] + ".png")

        # Load and process image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load ground truth mask
        gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        gt_mask = (gt_mask > 0).astype(np.uint8)

        # Model Prediction
        input_tensor = transform(image=image)["image"].unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            pred = torch.sigmoid(model(input_tensor)).cpu().numpy()[0, 0]
            pred_mask = (pred > 0.5).astype(np.uint8)

        # --- PLOTTING ---
        # Original Image
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(image)
        plt.title(f"Original: {img_name}")
        plt.axis('off')

        # Asli Mask (Ground Truth)
        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(gt_mask, cmap='gray')
        plt.title("Ground Truth (Asli)")
        plt.axis('off')

        # Model ka Mask (Prediction)
        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(pred_mask, cmap='jet', alpha=0.5) # Overlay effect
        plt.imshow(pred_mask, cmap='gray', alpha=0.5)
        plt.title("Model Prediction")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    visualize_preds(num_samples=4)


import os
import cv2
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from IPython.display import FileLink

# --- 1. LIBRARIES & INSTALLATION ---
try:
    import segmentation_models_pytorch as smp
except:
    os.system('pip install -q segmentation_models_pytorch albumentations')
    import segmentation_models_pytorch as smp

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torch.amp import GradScaler, autocast

# --- 2. CONFIGURATION ---
CFG = {
    "img_size": 768,        # Ultra resolution for small terrain details
    "batch_size": 4,        # B7 + 768 res ke liye 4 safe hai (OOM se bachne ke liye)
    "epochs": 40,           
    "patience": 8,          
    "lr": 1e-4,             
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "train_img": "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images",
    "train_mask": "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks",
    "test_dir": "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/test_images_padded",
    "model_path": "best_rugged_ultra.pth",
    "csv_path": "submission.csv",
    "threshold": 0.5        # Isse 0.45-0.55 ke beech adjust kar sakte hain
}

# --- 3. AUGMENTATIONS ---
train_transform = A.Compose([
    A.Resize(CFG["img_size"], CFG["img_size"]),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomRotate90(p=0.5),
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 50.0)),
        A.CLAHE(clip_limit=2.0),
        A.RandomBrightnessContrast(),
    ], p=0.4),
    A.Normalize(),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG["img_size"], CFG["img_size"]),
    A.Normalize(),
    ToTensorV2()
])

# --- 4. DATASET & LOSS ---
class OffroadDataset(Dataset):
    def __init__(self, images, img_dir, mask_dir, transform=None):
        self.images, self.img_dir, self.mask_dir, self.transform = images, img_dir, mask_dir, transform
    def __len__(self): return len(self.images)
    def __getitem__(self, idx):
        name = self.images[idx]
        img = cv2.cvtColor(cv2.imread(os.path.join(self.img_dir, name)), cv2.COLOR_BGR2RGB)
        mask_name = os.path.splitext(name)[0] + ".png"
        mask = cv2.imread(os.path.join(self.mask_dir, mask_name), cv2.IMREAD_GRAYSCALE)
        mask = (mask > 0).astype(np.float32)
        if self.transform:
            aug = self.transform(image=img, mask=mask)
            img, mask = aug['image'], aug['mask']
        return img, mask.unsqueeze(0)

class RuggedLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.dice = smp.losses.DiceLoss(mode='binary')
        self.focal = smp.losses.FocalLoss(mode='binary', gamma=2.0)
        self.bce = nn.BCEWithLogitsLoss()
    def forward(self, logits, targets):
        # Focus on pixel accuracy (BCE) and hard examples (Focal)
        return 0.4 * self.dice(logits, targets) + 0.4 * self.focal(logits, targets) + 0.2 * self.bce(logits, targets)

# --- 5. RLE ENCODER (Order 'F') ---
def rle_encode(mask01):
    pixels = mask01.flatten(order="F") 
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

# --- 6. MAIN PIPELINE ---
def main():
    # Setup Data
    all_imgs = sorted([f for f in os.listdir(CFG["train_img"]) if f.endswith(('.png', '.jpg'))])
    train_idx, val_idx = train_test_split(all_imgs, test_size=0.1, random_state=42)

    # Check if training is needed
    if os.path.exists(CFG["model_path"]):
        print(f"✅ Found existing model {CFG['model_path']}. Skipping training phase...")
        train_needed = False
    else:
        print("🚀 Starting Ultra Training (EfficientNet-B7)...")
        train_needed = True

    model = smp.UnetPlusPlus(encoder_name="efficientnet-b7", encoder_weights="imagenet", in_channels=3, classes=1).to(CFG["device"])

    if train_needed:
        train_loader = DataLoader(OffroadDataset(train_idx, CFG["train_img"], CFG["train_mask"], train_transform), 
                                  batch_size=CFG["batch_size"], shuffle=True, num_workers=4, pin_memory=True)
        val_loader = DataLoader(OffroadDataset(val_idx, CFG["train_img"], CFG["train_mask"], val_transform), 
                                batch_size=CFG["batch_size"], shuffle=False, num_workers=4)

        optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["lr"], weight_decay=1e-4)
        criterion = RuggedLoss()
        scaler = GradScaler()
        best_iou = 0
        patience_counter = 0

        for epoch in range(1, CFG["epochs"] + 1):
            model.train()
            pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
            for imgs, masks in pbar:
                imgs, masks = imgs.to(CFG["device"]), masks.to(CFG["device"])
                optimizer.zero_grad()
                with autocast(device_type='cuda'):
                    out = model(imgs)
                    loss = criterion(out, masks)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                
                # Memory management inside batch
                del imgs, masks, out, loss
            
            # Validation
            model.eval()
            v_iou = 0
            with torch.no_grad():
                for imgs, masks in val_loader:
                    imgs, masks = imgs.to(CFG["device"]), masks.to(CFG["device"])
                    out = (torch.sigmoid(model(imgs)) > CFG["threshold"]).float()
                    inter = (out * masks).sum()
                    union = (out + masks).sum() - inter
                    v_iou += (inter + 1e-7) / (union + 1e-7)
            
            avg_iou = (v_iou / len(val_loader)).item()
            print(f"📊 Epoch {epoch} Validation IoU: {avg_iou:.5f}")

            if avg_iou > best_iou:
                best_iou = avg_iou
                torch.save({'model_state_dict': model.state_dict(), 'iou': best_iou}, CFG["model_path"])
                print(f"⭐ Best Model Saved! IoU: {best_iou:.5f}")
                patience_counter = 0
            else:
                patience_counter += 1
            
            # GC & Cache Clear to prevent OOM
            gc.collect()
            torch.cuda.empty_cache()

            if patience_counter >= CFG["patience"]:
                print("🛑 Early Stopping triggered.")
                break

    # --- INFERENCE PHASE ---
    print("\n🎬 Starting Inference with TTA (Test Time Augmentation)...")
    checkpoint = torch.load(CFG["model_path"], map_location=CFG["device"])
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    test_imgs = sorted([f for f in os.listdir(CFG["test_dir"]) if f.endswith(('.png', '.jpg'))])
    results = []

    with torch.no_grad():
        for name in tqdm(test_imgs):
            img_path = os.path.join(CFG["test_dir"], name)
            img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
            
            # TTA: Original + Horizontal Flip
            t1 = val_transform(image=img)["image"].unsqueeze(0).to(CFG["device"])
            t2 = val_transform(image=cv2.flip(img, 1))["image"].unsqueeze(0).to(CFG["device"])
            
            p1 = torch.sigmoid(model(t1))
            p2 = torch.flip(torch.sigmoid(model(t2)), [3])
            
            # Average Prediction for higher stability
            final_p = ((p1 + p2) / 2).cpu().numpy()[0, 0]
            
            # Thresholding and Resizing back to competition size (960x540)
            mask01 = (cv2.resize(final_p, (960, 540), interpolation=cv2.INTER_NEAREST) > CFG["threshold"]).astype(np.uint8)
            
            results.append({"image_id": os.path.splitext(name)[0], "encoded_pixels": rle_encode(mask01)})
            
            # Frequent memory clearing during inference
            if len(results) % 50 == 0:
                gc.collect()
                torch.cuda.empty_cache()

    # Final Save
    df = pd.DataFrame(results)
    df.to_csv(CFG["csv_path"], index=False)
    print(f"🏁 Process Complete! submission.csv created.")
    
    # Download Link for the model
    display(FileLink(CFG["model_path"]))

if __name__ == "__main__":
    main()


import os
import cv2
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from IPython.display import FileLink

# --- 1. LIBRARIES & INSTALLATION ---
try:
    import segmentation_models_pytorch as smp
except:
    os.system('pip install -q segmentation_models_pytorch albumentations')
    import segmentation_models_pytorch as smp

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torch.amp import GradScaler, autocast

# --- 2. CONFIGURATION (Fixed for OOM) ---
CFG = {
    "img_size": 768,        
    "batch_size": 2,        # 4 se 2 kiya OOM bachane ke liye
    "grad_accum": 2,        # 2 batches ke baad update karega (Effective Batch Size = 4)
    "epochs": 40,           
    "patience": 8,          
    "lr": 1e-4,             
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "train_img": "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images",
    "train_mask": "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks",
    "test_dir": "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/test_images_padded",
    "model_path": "best_rugged_ultra.pth",
    "csv_path": "submission.csv",
    "threshold": 0.5        
}

# --- 3. AUGMENTATIONS ---
train_transform = A.Compose([
    A.Resize(CFG["img_size"], CFG["img_size"]),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomRotate90(p=0.5),
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 50.0)),
        A.CLAHE(clip_limit=2.0),
        A.RandomBrightnessContrast(),
    ], p=0.4),
    A.Normalize(),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG["img_size"], CFG["img_size"]),
    A.Normalize(),
    ToTensorV2()
])

# --- 4. DATASET & LOSS ---
class OffroadDataset(Dataset):
    def __init__(self, images, img_dir, mask_dir, transform=None):
        self.images, self.img_dir, self.mask_dir, self.transform = images, img_dir, mask_dir, transform
    def __len__(self): return len(self.images)
    def __getitem__(self, idx):
        name = self.images[idx]
        img_path = os.path.join(self.img_dir, name)
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        mask_name = os.path.splitext(name)[0] + ".png"
        mask = cv2.imread(os.path.join(self.mask_dir, mask_name), cv2.IMREAD_GRAYSCALE)
        mask = (mask > 0).astype(np.float32)
        if self.transform:
            aug = self.transform(image=img, mask=mask)
            img, mask = aug['image'], aug['mask']
        return img, mask.unsqueeze(0)

class RuggedLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.dice = smp.losses.DiceLoss(mode='binary')
        self.focal = smp.losses.FocalLoss(mode='binary', gamma=2.0)
        self.bce = nn.BCEWithLogitsLoss()
    def forward(self, logits, targets):
        return 0.4 * self.dice(logits, targets) + 0.4 * self.focal(logits, targets) + 0.2 * self.bce(logits, targets)

def rle_encode(mask01):
    pixels = mask01.flatten(order="F") 
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

# --- 6. MAIN PIPELINE ---
def main():
    # Setup Data
    all_imgs = sorted([f for f in os.listdir(CFG["train_img"]) if f.endswith(('.png', '.jpg'))])
    train_idx, val_idx = train_test_split(all_imgs, test_size=0.1, random_state=42)

    if os.path.exists(CFG["model_path"]):
        print(f"✅ Skipping training phase...")
        train_needed = False
    else:
        print("🚀 Starting Ultra Training (EfficientNet-B6 for Memory Efficiency)...")
        train_needed = True

    # B6 is used instead of B7 to avoid OOM while maintaining accuracy
    model = smp.UnetPlusPlus(encoder_name="efficientnet-b6", encoder_weights="imagenet", in_channels=3, classes=1).to(CFG["device"])

    if train_needed:
        train_loader = DataLoader(OffroadDataset(train_idx, CFG["train_img"], CFG["train_mask"], train_transform), 
                                  batch_size=CFG["batch_size"], shuffle=True, num_workers=2, pin_memory=True)
        val_loader = DataLoader(OffroadDataset(val_idx, CFG["train_img"], CFG["train_mask"], val_transform), 
                                batch_size=CFG["batch_size"], shuffle=False, num_workers=2)

        optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["lr"], weight_decay=1e-4)
        criterion = RuggedLoss()
        scaler = GradScaler()
        best_iou = 0
        patience_counter = 0

        for epoch in range(1, CFG["epochs"] + 1):
            model.train()
            pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
            optimizer.zero_grad() # Move outside inner loop for accumulation
            
            for i, (imgs, masks) in enumerate(pbar):
                imgs, masks = imgs.to(CFG["device"]), masks.to(CFG["device"])
                
                with autocast(device_type='cuda'):
                    out = model(imgs)
                    loss = criterion(out, masks)
                    loss = loss / CFG["grad_accum"] # Normalize loss
                
                scaler.scale(loss).backward()
                
                if (i + 1) % CFG["grad_accum"] == 0:
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
                
                del imgs, masks, out, loss
            
            # Validation
            model.eval()
            v_iou = 0
            with torch.no_grad():
                for imgs, masks in val_loader:
                    imgs, masks = imgs.to(CFG["device"]), masks.to(CFG["device"])
                    out = (torch.sigmoid(model(imgs)) > CFG["threshold"]).float()
                    inter = (out * masks).sum()
                    union = (out + masks).sum() - inter
                    v_iou += (inter + 1e-7) / (union + 1e-7)
            
            avg_iou = (v_iou / len(val_loader)).item()
            print(f"📊 Validation IoU: {avg_iou:.5f}")

            if avg_iou > best_iou:
                best_iou = avg_iou
                torch.save({'model_state_dict': model.state_dict(), 'iou': best_iou}, CFG["model_path"])
                patience_counter = 0
            else:
                patience_counter += 1
            
            gc.collect()
            torch.cuda.empty_cache()

            if patience_counter >= CFG["patience"]: break

    # --- INFERENCE ---
    print("\n🎬 Inference with TTA...")
    checkpoint = torch.load(CFG["model_path"], map_location=CFG["device"])
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    test_imgs = sorted([f for f in os.listdir(CFG["test_dir"]) if f.endswith(('.png', '.jpg'))])
    results = []
    with torch.no_grad():
        for name in tqdm(test_imgs):
            img = cv2.cvtColor(cv2.imread(os.path.join(CFG["test_dir"], name)), cv2.COLOR_BGR2RGB)
            t1 = val_transform(image=img)["image"].unsqueeze(0).to(CFG["device"])
            t2 = val_transform(image=cv2.flip(img, 1))["image"].unsqueeze(0).to(CFG["device"])
            p1 = torch.sigmoid(model(t1))
            p2 = torch.flip(torch.sigmoid(model(t2)), [3])
            final_p = ((p1 + p2) / 2).cpu().numpy()[0, 0]
            mask01 = (cv2.resize(final_p, (960, 540), interpolation=cv2.INTER_NEAREST) > CFG["threshold"]).astype(np.uint8)
            results.append({"image_id": os.path.splitext(name)[0], "encoded_pixels": rle_encode(mask01)})
            if len(results) % 100 == 0:
                torch.cuda.empty_cache()

    pd.DataFrame(results).to_csv(CFG["csv_path"], index=False)
    display(FileLink(CFG["model_path"]))

if __name__ == "__main__":
    main()


import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- CONFIGURATION ---
MODEL_PATH = "best_rugged_ultra.pth"
IMG_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images"
MASK_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def visualize_samples(num_samples=3):
    # 1. Model Load karein (EfficientNet-B6)
    model = smp.UnetPlusPlus(encoder_name="efficientnet-b6", encoder_weights=None, in_channels=3, classes=1).to(DEVICE)
    
    if not os.path.exists(MODEL_PATH):
        print(f"❌ Error: {MODEL_PATH} file nahi mili! Pehle check karein ki file saved hai ya nahi.")
        return

    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # Preprocessing transforms
    transform = A.Compose([
        A.Resize(768, 768),
        A.Normalize(),
        ToTensorV2()
    ])

    # Random images select karein
    all_imgs = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.png', '.jpg'))])
    indices = np.random.choice(len(all_imgs), num_samples, replace=False)

    plt.figure(figsize=(18, 6 * num_samples))

    for i, idx in enumerate(indices):
        img_name = all_imgs[idx]
        img_path = os.path.join(IMG_DIR, img_name)
        mask_path = os.path.join(MASK_DIR, os.path.splitext(img_name)[0] + ".png")

        # Load Image and Mask
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        gt_mask = (gt_mask > 0).astype(np.uint8)

        # Model Prediction
        input_tensor = transform(image=image)["image"].unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            pred = torch.sigmoid(model(input_tensor)).cpu().numpy()[0, 0]
            # Resize back for visualization
            pred = cv2.resize(pred, (image.shape[1], image.shape[0]))
            pred_mask = (pred > 0.5).astype(np.uint8)

        # --- PLOT ---
        # 1. Original Image
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(image)
        plt.title(f"Original Image: {img_name}")
        plt.axis('off')

        # 2. Ground Truth (Asli Mask)
        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(gt_mask, cmap='gray')
        plt.title("Ground Truth (Asli Label)")
        plt.axis('off')

        # 3. Model Prediction (Predicted Mask)
        plt.subplot(num_samples, 3, i*3 + 3)
        # Prediction ko original image ke upar overlay dikhate hain (Green color)
        overlay = image.copy()
        overlay[pred_mask == 1] = [0, 255, 0] # Predicted area green dikhega
        plt.imshow(cv2.addWeighted(image, 0.6, overlay, 0.4, 0))
        plt.title("Model Prediction (Overlay)")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    visualize_samples(num_samples=3)


import torch
import pandas as pd
import os
import cv2
from tqdm import tqdm
import numpy as np
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- SETTINGS ---
MODEL_PATH = "best_rugged_ultra.pth"
TEST_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/test_images_padded"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Same transforms as training
val_transform = A.Compose([
    A.Resize(768, 768),
    A.Normalize(),
    ToTensorV2()
])

def rle_encode(mask01):
    pixels = mask01.flatten(order="F") 
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

# 1. Load Model
model = smp.UnetPlusPlus(encoder_name="efficientnet-b6", encoder_weights=None, in_channels=3, classes=1).to(DEVICE)
checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# 2. Inference
test_imgs = sorted([f for f in os.listdir(TEST_DIR) if f.endswith(('.png', '.jpg'))])
results = []

print("🎬 Generating submission from current checkpoint...")
with torch.no_grad():
    for name in tqdm(test_imgs):
        img = cv2.cvtColor(cv2.imread(os.path.join(TEST_DIR, name)), cv2.COLOR_BGR2RGB)
        
        # Simple Prediction (No TTA for speed)
        t1 = val_transform(image=img)["image"].unsqueeze(0).to(DEVICE)
        pred = torch.sigmoid(model(t1)).cpu().numpy()[0, 0]
        
        # Resize to Original Competition Size
        mask01 = (cv2.resize(pred, (960, 540), interpolation=cv2.INTER_NEAREST) > 0.5).astype(np.uint8)
        results.append({"image_id": os.path.splitext(name)[0], "encoded_pixels": rle_encode(mask01)})

# 3. Save CSV
pd.DataFrame(results).to_csv("submission.csv", index=False)
print("🏁 submission.csv is ready! Ab aap 'Submit' button daba sakte hain.")


import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- CONFIG ---
MODEL_PATH = "best_rugged_ultra.pth"
IMG_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images"
MASK_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NEW_THRESHOLD = 0.85  # Isse badha diya hai taaki sky/noise hat jaye

def show_results(num_samples=2):
    model = smp.UnetPlusPlus(encoder_name="efficientnet-b6", encoder_weights=None, in_channels=3, classes=1).to(DEVICE)
    if not os.path.exists(MODEL_PATH):
        print("❌ Model file nahi mili!")
        return
    
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    transform = A.Compose([A.Resize(768, 768), A.Normalize(), ToTensorV2()])
    all_imgs = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.png', '.jpg'))])
    indices = np.random.choice(len(all_imgs), num_samples)

    plt.figure(figsize=(20, 10))
    for i, idx in enumerate(indices):
        name = all_imgs[idx]
        img = cv2.cvtColor(cv2.imread(os.path.join(IMG_DIR, name)), cv2.COLOR_BGR2RGB)
        gt = (cv2.imread(os.path.join(MASK_DIR, os.path.splitext(name)[0]+".png"), 0) > 0).astype(int)

        tensor = transform(image=img)["image"].unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            probs = torch.sigmoid(model(tensor)).cpu().numpy()[0, 0]
            probs = cv2.resize(probs, (img.shape[1], img.shape[0]))
            pred_mask = (probs > NEW_THRESHOLD).astype(np.uint8)

        # Plotting
        plt.subplot(num_samples, 4, i*4 + 1); plt.imshow(img); plt.title("Original"); plt.axis('off')
        plt.subplot(num_samples, 4, i*4 + 2); plt.imshow(gt, cmap='gray'); plt.title("Ground Truth"); plt.axis('off')
        plt.subplot(num_samples, 4, i*4 + 3); plt.imshow(pred_mask, cmap='gray'); plt.title(f"Pred (Thresh {NEW_THRESHOLD})"); plt.axis('off')
        plt.subplot(num_samples, 4, i*4 + 4); plt.imshow(probs, cmap='jet'); plt.title("Confidence Heatmap"); plt.axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    show_results()


import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- CONFIG ---
MODEL_PATH = "best_rugged_ultra.pth"
IMG_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images"
MASK_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- AUTO-REPAIR SETTINGS ---
# Agar model sab kuch white dikha raha hai, toh threshold ko 0.9 tak le jayein
DYNAMIC_THRESHOLD = 0.85 

def show_corrected_results(num_samples=2):
    # Model architecture must match your training (EfficientNet-B6)
    model = smp.UnetPlusPlus(encoder_name="efficientnet-b6", encoder_weights=None, in_channels=3, classes=1).to(DEVICE)
    
    if not os.path.exists(MODEL_PATH):
        print("❌ Model file nahi mili! Check /kaggle/working/ directory.")
        return
    
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    transform = A.Compose([
        A.Resize(768, 768),
        A.Normalize(),
        ToTensorV2()
    ])
    
    all_imgs = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.png', '.jpg'))])
    indices = np.random.choice(len(all_imgs), num_samples)

    plt.figure(figsize=(20, 12))
    
    for i, idx in enumerate(indices):
        name = all_imgs[idx]
        img_path = os.path.join(IMG_DIR, name)
        mask_path = os.path.join(MASK_DIR, os.path.splitext(name)[0] + ".png")
        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gt = (cv2.imread(mask_path, 0) > 0).astype(np.uint8)

        # Inference
        tensor = transform(image=img)["image"].unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            probs = torch.sigmoid(model(tensor)).cpu().numpy()[0, 0]
            probs = cv2.resize(probs, (img.shape[1], img.shape[0]))
            
            # Apply dynamic threshold to fix over-segmentation
            pred_mask = (probs > DYNAMIC_THRESHOLD).astype(np.uint8)

        # Plotting
        # 1. Original
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(img)
        plt.title(f"Original Image")
        plt.axis('off')

        # 2. Ground Truth (Expected)
        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(gt, cmap='gray')
        plt.title("Ground Truth (Target)")
        plt.axis('off')

        # 3. Corrected Prediction (Binary Mask)
        plt.subplot(num_samples, 3, i*3 + 3)
        # Overlay dikhate hain: Red color for prediction error
        plt.imshow(pred_mask, cmap='jet') 
        plt.title(f"Predicted Mask (Thresh {DYNAMIC_THRESHOLD})")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    show_corrected_results()


import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- CONFIG ---
MODEL_PATH = "best_rugged_ultra.pth"
IMG_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images"
# MASK_DIR sirf comparison ke liye hai
MASK_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks" 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def smart_inference(model, tensor):
    with torch.no_grad():
        probs = torch.sigmoid(model(tensor)).cpu().numpy()[0, 0]
        
    # --- SMART FIX LOGIC ---
    # Calculate density (Kitna percent area white hai)
    density = np.mean(probs)
    
    # Agar model bolta hai ki 50% se zyada image terrain hai, toh ye galat hai.
    # Offroad terrain usually sparse hota hai. Isliye hum isse INVERT kar denge.
    if density > 0.50: 
        probs = 1.0 - probs  # Invert logic
        
    return probs

def show_fixed_results(num_samples=3):
    # Model Load
    model = smp.UnetPlusPlus(encoder_name="efficientnet-b6", encoder_weights=None, in_channels=3, classes=1).to(DEVICE)
    if not os.path.exists(MODEL_PATH):
        print("❌ Model file nahi mili!")
        return
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    transform = A.Compose([A.Resize(768, 768), A.Normalize(), ToTensorV2()])
    all_imgs = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.png', '.jpg'))])
    
    # Randomly pick images
    indices = np.random.choice(len(all_imgs), num_samples)

    plt.figure(figsize=(18, 6 * num_samples))
    
    for i, idx in enumerate(indices):
        name = all_imgs[idx]
        img_path = os.path.join(IMG_DIR, name)
        mask_path = os.path.join(MASK_DIR, os.path.splitext(name)[0] + ".png")
        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Ground Truth (Using your specific 27/39 logic just for visualization)
        gt_raw = cv2.imread(mask_path, 0)
        # Check specific values
        gt = np.isin(gt_raw, [27, 39]).astype(np.uint8)

        # Smart Inference
        tensor = transform(image=img)["image"].unsqueeze(0).to(DEVICE)
        probs = smart_inference(model, tensor)
        
        # Resize back
        probs = cv2.resize(probs, (img.shape[1], img.shape[0]))
        pred_mask = (probs > 0.5).astype(np.uint8)

        # --- PLOTS ---
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(img)
        plt.title(f"Original: {name}")
        plt.axis('off')

        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(gt, cmap='gray')
        plt.title("Ground Truth (Corrected 27/39)")
        plt.axis('off')

        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(pred_mask, cmap='gray')
        # Agar invert logic laga hoga toh title mein dikhega
        mean_val = np.mean(pred_mask)
        status = " (Inverted)" if mean_val < 0.5 and np.mean(torch.sigmoid(model(tensor)).cpu().numpy()) > 0.5 else ""
        plt.title(f"Smart Prediction{status}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    show_fixed_results()
    


import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- CONFIG ---
MODEL_PATH = "best_rugged_ultra.pth"
IMG_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images"
# Mask dir sirf ground truth compare karne ke liye
MASK_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks" 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def smart_predict_viz(model, tensor):
    with torch.no_grad():
        # .detach() zaroori hai error hatane ke liye
        logits = model(tensor)
        probs = torch.sigmoid(logits).detach().cpu().numpy()[0, 0]
    
    # --- SMART FIX (INVERT LOGIC) ---
    # Agar mean value > 0.5 hai (yani puri image white hai), to iska matlab model ulat gaya hai.
    if np.mean(probs) > 0.5:
        probs = 1.0 - probs  # Invert kar do (White -> Black, Black -> White)
        
    return probs

def show_final_results(num_samples=3):
    # Model Load
    model = smp.UnetPlusPlus(encoder_name="efficientnet-b6", encoder_weights=None, in_channels=3, classes=1).to(DEVICE)
    
    if not os.path.exists(MODEL_PATH):
        print(f"❌ Error: {MODEL_PATH} nahi mili!")
        return

    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    transform = A.Compose([A.Resize(768, 768), A.Normalize(), ToTensorV2()])
    
    # Random Images
    all_imgs = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.png', '.jpg'))])
    indices = np.random.choice(len(all_imgs), num_samples)

    plt.figure(figsize=(18, 6 * num_samples))
    
    for i, idx in enumerate(indices):
        name = all_imgs[idx]
        img_path = os.path.join(IMG_DIR, name)
        mask_path = os.path.join(MASK_DIR, os.path.splitext(name)[0] + ".png")
        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Ground Truth (Aapka logic: 27/39 wala agar raw values wahi hain)
        gt_raw = cv2.imread(mask_path, 0)
        gt = (gt_raw > 0).astype(np.uint8) # Simple binary for viz

        # Inference
        tensor = transform(image=img)["image"].unsqueeze(0).to(DEVICE)
        
        # Smart Prediction call
        probs = smart_predict_viz(model, tensor)
        
        # Resize back to image shape
        probs = cv2.resize(probs, (img.shape[1], img.shape[0]))
        pred_mask = (probs > 0.5).astype(np.uint8)

        # PLOT
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(img)
        plt.title(f"Original: {name}")
        plt.axis('off')

        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(gt, cmap='gray')
        plt.title("Ground Truth")
        plt.axis('off')

        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(pred_mask, cmap='gray')
        plt.title("Smart Prediction (Inverted if needed)")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    show_final_results()
    


import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- CONFIG ---
MODEL_PATH = "best_rugged_ultra.pth"
IMG_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images"
MASK_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks" 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# AGAR COLOR ULTA DIKHE TOH ISSE True KAREIN
INVERT_COLORS = False 

def show_results(num_samples=3):
    # Model Load
    model = smp.UnetPlusPlus(encoder_name="efficientnet-b6", encoder_weights=None, in_channels=3, classes=1).to(DEVICE)
    
    if not os.path.exists(MODEL_PATH):
        print(f"❌ Error: {MODEL_PATH} nahi mili!")
        return

    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    transform = A.Compose([A.Resize(768, 768), A.Normalize(), ToTensorV2()])
    all_imgs = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.png', '.jpg'))])
    indices = np.random.choice(len(all_imgs), num_samples)

    plt.figure(figsize=(18, 6 * num_samples))
    
    for i, idx in enumerate(indices):
        name = all_imgs[idx]
        img_path = os.path.join(IMG_DIR, name)
        mask_path = os.path.join(MASK_DIR, os.path.splitext(name)[0] + ".png")
        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Ground Truth check
        gt = cv2.imread(mask_path, 0)
        gt = (gt > 0).astype(np.uint8)

        # Inference
        tensor = transform(image=img)["image"].unsqueeze(0).to(DEVICE)
        
        with torch.no_grad():
            logits = model(tensor)
            # FIX: Added .detach() here to solve RuntimeError
            probs = torch.sigmoid(logits).detach().cpu().numpy()[0, 0]
        
        # Resize back to original
        probs = cv2.resize(probs, (img.shape[1], img.shape[0]))
        
        # Manual Invert Check
        if INVERT_COLORS:
            probs = 1.0 - probs
            
        pred_mask = (probs > 0.5).astype(np.uint8)

        # PLOT
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(img)
        plt.title(f"Original: {name}")
        plt.axis('off')

        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(gt, cmap='gray')
        plt.title("Ground Truth (White=Terrain)")
        plt.axis('off')

        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(pred_mask, cmap='gray')
        plt.title(f"Prediction {'(Inverted)' if INVERT_COLORS else '(Raw)'}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    show_results()


import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os

# --- SETTINGS ---
MODEL_PATH = "best_rugged_ultra.pth"
TEST_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/test_images_padded"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def show_samples(num_samples=3):
    # 1. Load Model
    model = smp.UnetPlusPlus(
        encoder_name="efficientnet-b6", 
        encoder_weights=None, 
        in_channels=3, 
        classes=1
    ).to(DEVICE)
    
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    transform = A.Compose([
        A.Resize(768, 768), 
        A.Normalize(), 
        ToTensorV2()
    ])

    test_imgs = [f for f in os.listdir(TEST_DIR) if f.endswith(('.png', '.jpg'))][:num_samples]
    
    plt.figure(figsize=(18, 5 * num_samples))

    for i, name in enumerate(test_imgs):
        img_path = os.path.join(TEST_DIR, name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Inference
        tensor = transform(image=img)["image"].unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            output = model(tensor)
            # .detach() add kiya hai error se bachne ke liye
            pred_raw = torch.sigmoid(output).detach().cpu().numpy()[0, 0]
        
        # Inversion Logic
        pred_inverted = 1.0 - pred_raw
        
        # Thresholding
        mask_raw = (pred_raw > 0.5).astype(np.uint8)
        mask_inverted = (pred_inverted > 0.5).astype(np.uint8)

        # Plotting
        # 1. Original
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(img)
        plt.title(f"Original: {name}")
        plt.axis('off')

        # 2. Raw Prediction (Jo pehle ulta tha)
        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(mask_raw, cmap='gray')
        plt.title("Raw Model Output (Before Invert)")
        plt.axis('off')

        # 3. Final Inverted Prediction (Jo hum submit karenge)
        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(mask_inverted, cmap='gray')
        plt.title("FINAL (Inverted - For Submission)")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    show_samples()


import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os

# --- SETTINGS ---
MODEL_PATH = "best_rugged_ultra.pth"
TEST_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/test_images_padded"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def show_samples(num_samples=5):
    # 1. Load Model
    model = smp.UnetPlusPlus(
        encoder_name="efficientnet-b6", 
        encoder_weights=None, 
        in_channels=3, 
        classes=1
    ).to(DEVICE)
    
    if not os.path.exists(MODEL_PATH):
        print(f"❌ Error: {MODEL_PATH} nahi mili!")
        return

    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    transform = A.Compose([
        A.Resize(768, 768), 
        A.Normalize(), 
        ToTensorV2()
    ])

    # Saari images ki list lekar random ya first n samples uthayenge
    all_test_imgs = sorted([f for f in os.listdir(TEST_DIR) if f.endswith(('.png', '.jpg'))])
    test_imgs = all_test_imgs[:num_samples]
    
    if len(test_imgs) == 0:
        print("❌ Test directory mein koi images nahi mili!")
        return

    plt.figure(figsize=(18, 4 * num_samples))

    for i, name in enumerate(test_imgs):
        img_path = os.path.join(TEST_DIR, name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Inference
        tensor = transform(image=img)["image"].unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            output = model(tensor)
            pred_raw = torch.sigmoid(output).detach().cpu().numpy()[0, 0]
        
        # Inversion Logic
        pred_inverted = 1.0 - pred_raw
        
        # Thresholding
        mask_raw = (pred_raw > 0.5).astype(np.uint8)
        mask_inverted = (pred_inverted > 0.5).astype(np.uint8)

        # Plotting
        # 1. Original
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(img)
        plt.title(f"Original: {name}")
        plt.axis('off')

        # 2. Raw Prediction
        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(mask_raw, cmap='gray')
        plt.title("Raw (Before Invert)")
        plt.axis('off')

        # 3. Final Inverted Prediction
        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(mask_inverted, cmap='gray')
        plt.title("FINAL (Inverted)")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    # Yahan number badha kar aap aur bhi zyada images dekh sakte hain
    show_samples(num_samples=5)


import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- CONFIG ---
MODEL_PATH = "best_rugged_ultra.pth"
IMG_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images"
MASK_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks" 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Agar terrain (white) aur background (black) ulte dikhein toh True karein
INVERT_COLORS = False 

def visualize_model_performance(num_samples=3):
    # 1. Model Setup (Unet++ with EfficientNet-B6 as per your training)
    model = smp.UnetPlusPlus(
        encoder_name="efficientnet-b6", 
        encoder_weights=None, 
        in_channels=3, 
        classes=1
    ).to(DEVICE)
    
    if not os.path.exists(MODEL_PATH):
        print(f"❌ Error: {MODEL_PATH} file nahi mili! Check karein ki file upload ho gayi hai.")
        return

    # Load Weights
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    print(f"✅ Model loaded successfully from {MODEL_PATH}")

    # 2. Transforms
    transform = A.Compose([
        A.Resize(768, 768), 
        A.Normalize(), 
        ToTensorV2()
    ])

    # 3. Data Selection
    all_imgs = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.png', '.jpg'))])
    indices = np.random.choice(len(all_imgs), num_samples, replace=False)

    plt.figure(figsize=(20, 6 * num_samples))
    
    for i, idx in enumerate(indices):
        name = all_imgs[idx]
        img_path = os.path.join(IMG_DIR, name)
        mask_path = os.path.join(MASK_DIR, os.path.splitext(name)[0] + ".png")
        
        # Load Image
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Load Ground Truth
        gt = cv2.imread(mask_path, 0)
        gt = (gt > 0).astype(np.uint8)

        # Inference
        tensor = transform(image=img_rgb)["image"].unsqueeze(0).to(DEVICE)
        
        with torch.no_grad():
            logits = model(tensor)
            probs = torch.sigmoid(logits).detach().cpu().numpy()[0, 0]
        
        # Resize to match original for side-by-side comparison
        probs_res = cv2.resize(probs, (img_rgb.shape[1], img_rgb.shape[0]))
        
        if INVERT_COLORS:
            probs_res = 1.0 - probs_res
            
        pred_mask = (probs_res > 0.5).astype(np.uint8)

        # --- PLOTTING ---
        # 1. Original
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(img_rgb)
        plt.title(f"Original: {name}")
        plt.axis('off')

        # 2. Ground Truth
        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(gt, cmap='gray')
        plt.title("Ground Truth (Target)")
        plt.axis('off')

        # 3. Model Prediction Overlay
        plt.subplot(num_samples, 3, i*3 + 3)
        # Prediction ko original image par overlay karke dikhayenge (Green Tint)
        overlay = img_rgb.copy()
        overlay[pred_mask == 1] = [0, 255, 0] # Predicted terrain in Green
        combined = cv2.addWeighted(img_rgb, 0.6, overlay, 0.4, 0)
        
        plt.imshow(combined)
        plt.title(f"Prediction (Green = Terrain)")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    visualize_model_performance(num_samples=3)


import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- CONFIG ---
MODEL_PATH = "best_rugged_ultra.pth"
IMG_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images"
MASK_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks" 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# AGAR COLOR ULTA DIKHE TOH ISSE True KAREIN
INVERT_COLORS = False 

def show_results(num_samples=3):
    # Model Load
    model = smp.UnetPlusPlus(
        encoder_name="efficientnet-b6", 
        encoder_weights=None, 
        in_channels=3, 
        classes=1
    ).to(DEVICE)
    
    if not os.path.exists(MODEL_PATH):
        print(f"❌ Error: {MODEL_PATH} nahi mili!")
        return

    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    transform = A.Compose([A.Resize(768, 768), A.Normalize(), ToTensorV2()])
    
    # Image lists
    all_imgs = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.png', '.jpg'))])
    indices = np.random.choice(len(all_imgs), num_samples)

    plt.figure(figsize=(18, 6 * num_samples))
    
    for i, idx in enumerate(indices):
        name = all_imgs[idx]
        img_path = os.path.join(IMG_DIR, name)
        mask_path = os.path.join(MASK_DIR, os.path.splitext(name)[0] + ".png")
        
        # Load Image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Ground Truth check
        gt = cv2.imread(mask_path, 0)
        if gt is None:
            # Fallback agar mask extension alag ho
            mask_path = os.path.join(MASK_DIR, os.path.splitext(name)[0] + ".jpg")
            gt = cv2.imread(mask_path, 0)
            
        gt = (gt > 0).astype(np.uint8) if gt is not None else np.zeros((img.shape[0], img.shape[1]))

        # Inference
        tensor = transform(image=img)["image"].unsqueeze(0).to(DEVICE)
        
        with torch.no_grad():
            logits = model(tensor)
            # FIX: Added .detach() here to solve RuntimeError
            probs = torch.sigmoid(logits).detach().cpu().numpy()[0, 0]
        
        # Resize back to original
        probs = cv2.resize(probs, (img.shape[1], img.shape[0]))
        
        # Manual Invert Check
        if INVERT_COLORS:
            probs = 1.0 - probs
            
        pred_mask = (probs > 0.5).astype(np.uint8)

        # PLOT
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(img)
        plt.title(f"Original: {name}")
        plt.axis('off')

        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(gt, cmap='gray')
        plt.title("Ground Truth (White=Terrain)")
        plt.axis('off')

        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(pred_mask, cmap='gray')
        plt.title(f"Prediction {'(Inverted)' if INVERT_COLORS else '(Raw)'}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    show_results()


import os
import glob
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

# --- CONFIG ---
IN_DIR = "test_images_padded"        # Jahan grayscale masks hain
IMG_DIR = "test_images"             # Jahan original images rakhi hain (.jpg ya .png)
VIZ_DIR = "viz_output"               # Jahan results save honge
OUT_CSV = "submission.csv"
H, W = 540, 960

# 27: Mud/Soil, 39: Grass/Vegetation
FG_CLASSES = {27, 39}

# Create visualization directory
os.makedirs(VIZ_DIR, exist_ok=True)

def rle_encode(mask01):
    """Kaggle specific RLE encoding (column-major)."""
    pixels = mask01.flatten(order="F")
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

def create_visual(original_img, mask01):
    """Original aur Mask ko side-by-side dikhane ke liye."""
    # 1. Mask ko Green color mein convert karein
    green_mask = np.zeros_like(original_img)
    green_mask[:, :, 1] = mask01 * 255  # Green channel set kiya
    
    # 2. Overlay banayein (Image + Green Mask)
    overlay = cv2.addWeighted(original_img, 0.7, green_mask, 0.3, 0)
    
    # 3. Mask ko display ke liye 3-channel grayscale banayein
    binary_display = cv2.cvtColor(mask01 * 255, cv2.COLOR_GRAY2BGR)
    
    # 4. Sabko horizontal stack karein: [Original | Mask | Overlay]
    combined = np.hstack([original_img, binary_display, overlay])
    return combined

def generate_final_submission_with_viz():
    rows = []
    paths = sorted(glob.glob(os.path.join(IN_DIR, "*.png")))
    
    if len(paths) == 0:
        print(f"❌ Error: {IN_DIR} mein koi masks nahi mile!")
        return

    print(f"🚀 Processing {len(paths)} images with visualization...")

    for p in tqdm(paths):
        # 1. Load Mask
        m = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        if m is None: continue

        # 2. Process Binary Mask
        mask01 = np.isin(m, list(FG_CLASSES)).astype(np.uint8)
        if mask01.shape != (H, W):
            mask01 = cv2.resize(mask01, (W, H), interpolation=cv2.INTER_NEAREST)

        # 3. RLE Encoding for CSV
        image_id = os.path.splitext(os.path.basename(p))[0]
        rle = rle_encode(mask01)
        rows.append({"image_id": image_id, "encoded_pixels": rle})

        # 4. Visualization Logic
        # Koshish karein ki original image mile (jpg/png check)
        img_path = os.path.join(IMG_DIR, f"{image_id}.jpg")
        if not os.path.exists(img_path):
            img_path = os.path.join(IMG_DIR, f"{image_id}.png")

        if os.path.exists(img_path):
            orig = cv2.imread(img_path)
            if orig is not None:
                orig = cv2.resize(orig, (W, H))
                viz_result = create_visual(orig, mask01)
                cv2.imwrite(os.path.join(VIZ_DIR, f"viz_{image_id}.jpg"), viz_result)
        
    # Save CSV
    df = pd.DataFrame(rows)
    df.to_csv(OUT_CSV, index=False)
    print(f"\n✅ Done! Check '{VIZ_DIR}' folder for visual results.")
    print(f"✅ Submission saved to: {OUT_CSV}")

if __name__ == "__main__":
    generate_final_submission_with_viz()


import os
import cv2
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

# --- CONFIG ---
MODEL_PATH = "best_rugged_ultra.pth"
# Yahan TEST images ka path dalein
TEST_IMG_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/test_images_padded" 
OUT_CSV = "submission.csv"
VISUAL_OUT = "prediction_check.png" # Visual results yahan save honge
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
H, W = 540, 960  # Kaggle expected resolution (Submission size)

def rle_encode(mask01):
    """Kaggle specific RLE encoding (column-major)."""
    pixels = mask01.flatten(order="F")
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

def generate_submission():
    # 1. Model Setup
    model = smp.UnetPlusPlus(
        encoder_name="efficientnet-b6", 
        encoder_weights=None, 
        in_channels=3, 
        classes=1
    ).to(DEVICE)
    
    if not os.path.exists(MODEL_PATH):
        print(f"❌ Error: Model file {MODEL_PATH} nahi mili!")
        return

    print(f"🔄 Loading model from {MODEL_PATH}...")
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    state_dict = checkpoint['model_state_dict'] if 'model_state_dict' in checkpoint else checkpoint
    model.load_state_dict(state_dict)
    model.eval()

    # 2. Transform Setup (Inference resize 768)
    transform = A.Compose([
        A.Resize(768, 768), 
        A.Normalize(), 
        ToTensorV2()
    ])
    
    if not os.path.exists(TEST_IMG_DIR):
        print(f"❌ Error: Test image directory {TEST_IMG_DIR} nahi mili!")
        return
        
    all_imgs = sorted([f for f in os.listdir(TEST_IMG_DIR) if f.endswith(('.png', '.jpg'))])
    print(f"🚀 Found {len(all_imgs)} images. Starting Inference...")

    rows = []
    visual_samples = [] # Visual results store karne ke liye

    # 4. Inference Loop
    with torch.no_grad():
        for i, name in enumerate(tqdm(all_imgs)):
            img_path = os.path.join(TEST_IMG_DIR, name)
            
            # Load Image
            img_bgr = cv2.imread(img_path)
            if img_bgr is None: continue
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            
            # Preprocess
            input_tensor = transform(image=img_rgb)["image"].unsqueeze(0).to(DEVICE)
            
            # Predict
            logits = model(input_tensor)
            probs = torch.sigmoid(logits).detach().cpu().numpy()[0, 0]
            
            # Resize back to Submission size
            probs_resized = cv2.resize(probs, (W, H), interpolation=cv2.INTER_LINEAR)
            mask01 = (probs_resized > 0.5).astype(np.uint8)
            
            # RLE Encode
            rle = rle_encode(mask01)
            image_id = os.path.splitext(name)[0]
            rows.append({"image_id": image_id, "encoded_pixels": rle})

            # Visual check ke liye pehli 3 images save karein
            if i < 3:
                visual_samples.append((img_rgb, mask01, name))

    # 5. Visual Result Save Karein (Matplotlib)
    if visual_samples:
        print(f"🖼️ Saving visual samples to {VISUAL_OUT}...")
        plt.figure(figsize=(15, 5 * len(visual_samples)))
        for idx, (original, pred, title) in enumerate(visual_samples):
            # Original Image
            plt.subplot(len(visual_samples), 2, idx*2 + 1)
            plt.imshow(original)
            plt.title(f"Original: {title}")
            plt.axis('off')
            
            # Predicted Mask
            plt.subplot(len(visual_samples), 2, idx*2 + 2)
            plt.imshow(pred, cmap='gray')
            plt.title("Model Prediction (Terrain)")
            plt.axis('off')
        
        plt.tight_layout()
        plt.savefig(VISUAL_OUT)
        plt.close()

    # 6. CSV Save Karein
    df = pd.DataFrame(rows)
    df.to_csv(OUT_CSV, index=False)
    
    print(f"\n✅ Success!")
    print(f"📂 CSV Saved: {OUT_CSV}")
    print(f"🖼️ Visual Check Saved: {VISUAL_OUT}")
    print(df.head())

if __name__ == "__main__":
    generate_submission()


import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader
import random

# --- CONFIGURATION ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "best_rugged_ultra.pth" # Aapki saved file ka naam
ENCODER = 'efficientnet-b6'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = 1 # Binary segmentation ke liye 1, multiclass ke liye badal dein

def visualize_predictions(model, dataset, num_samples=5):
    """
    Train dataset se random samples lekar original vs predicted dikhane ke liye function.
    """
    model.eval()
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))
    
    # Random indices select karna
    indices = random.sample(range(len(dataset)), num_samples)
    
    for i, idx in enumerate(indices):
        # Data load karna
        image, mask = dataset[idx]
        
        # Model prediction ke liye image prepare karna (Batch dimension add karna)
        input_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
        
        with torch.no_grad():
            output = model(input_tensor)
            # Sigmoid apply karke thresholding (0.5) karna
            pr_mask = torch.sigmoid(output).cpu().numpy().squeeze()
            pr_mask = (pr_mask > 0.5).astype(np.uint8)

        # Plotting
        # 1. Original Image (Transposing back to HWC for matplotlib)
        img_display = np.transpose(image, (1, 2, 0))
        # Agar image normalized hai, toh use wapas 0-1 range mein laana pad sakta hai
        img_display = (img_display - img_display.min()) / (img_display.max() - img_display.min())
        
        axes[i, 0].imshow(img_display)
        axes[i, 0].set_title(f"Sample {idx}: Original Image")
        axes[i, 0].axis('off')

        # 2. Ground Truth Mask
        axes[i, 1].imshow(mask.squeeze(), cmap='gray')
        axes[i, 1].set_title("Ground Truth (Original Mask)")
        axes[i, 1].axis('off')

        # 3. Predicted Mask
        axes[i, 2].imshow(pr_mask, cmap='jet')
        axes[i, 2].set_title("Model Prediction")
        axes[i, 2].axis('off')

    plt.tight_layout()
    plt.show()

# --- MAIN EXECUTION ---
def run_visualization(train_dataset):
    # 1. Model define aur load karein
    model = smp.UnetPlusPlus(
        encoder_name=ENCODER,
        encoder_weights=ENCODER_WEIGHTS,
        in_channels=3,
        classes=CLASSES,
    ).to(DEVICE)

    try:
        model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
        print("✅ Model weights successfully loaded!")
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return

    # 2. Visualization function call karein
    # Maan lijiye aapka 'train_dataset' object pehle se defined hai
    visualize_predictions(model, train_dataset, num_samples=3)

# Note: Is script ko chalane ke liye aapko apna 'train_dataset' pass karna hoga.
# Example: run_visualization(my_train_data)


import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
import random

# --- CONFIGURATION ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "best_rugged_ultra.pth" 
ENCODER = 'efficientnet-b6'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = 1 

def visualize_predictions(model, dataset, num_samples=3):
    model.eval()
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))
    
    # Randomly select samples
    indices = random.sample(range(len(dataset)), num_samples)
    
    for i, idx in enumerate(indices):
        # 1. Get Data
        image, mask = dataset[idx] # Expecting image: [C, H, W]
        
        # 2. Prepare for Model
        # Agar image tensor nahi hai toh convert karein
        if not isinstance(image, torch.Tensor):
            input_tensor = torch.from_numpy(image).float().to(DEVICE).unsqueeze(0)
        else:
            input_tensor = image.to(DEVICE).unsqueeze(0)
            
        # 3. Predict
        with torch.no_grad():
            output = model(input_tensor)
            pr_mask = torch.sigmoid(output).cpu().numpy().squeeze()
            pr_mask = (pr_mask > 0.5).astype(np.uint8)

        # 4. Display Logic
        # Convert image back to [H, W, C] for plotting
        if isinstance(image, torch.Tensor):
            img_display = image.permute(1, 2, 0).cpu().numpy()
        else:
            img_display = np.transpose(image, (1, 2, 0))

        # Denormalize image for visualization
        img_display = (img_display - img_display.min()) / (img_display.max() - img_display.min() + 1e-7)
        
        # Plot Original
        axes[i, 0].imshow(img_display)
        axes[i, 0].set_title(f"Sample {idx}: Original")
        axes[i, 0].axis('off')

        # Plot Ground Truth
        # Mask agar tensor hai toh numpy karein
        gt_mask = mask.cpu().numpy() if isinstance(mask, torch.Tensor) else mask
        axes[i, 1].imshow(gt_mask.squeeze(), cmap='gray')
        axes[i, 1].set_title("Ground Truth")
        axes[i, 1].axis('off')

        # Plot Prediction
        axes[i, 2].imshow(pr_mask, cmap='jet')
        axes[i, 2].set_title("Prediction")
        axes[i, 2].axis('off')

    plt.tight_layout()
    plt.show()

# --- RUNNING THE CODE ---
# Step 1: Initialize Model
model = smp.UnetPlusPlus(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    in_channels=3,
    classes=CLASSES,
).to(DEVICE)

# Step 2: Load Weights
try:
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    print("✅ Model loaded successfully!")
except FileNotFoundError:
    print(f"❌ Error: {MODEL_PATH} file nahi mili. Path check karein.")
except Exception as e:
    print(f"❌ Error: {e}")

# Step 3: Call the function
# YAHA DHAYAN DEIN: 'train_dataset' ki jagah wo variable likhein 
# jisme aapka training data stored hai (e.g., train_ds, dataset_train etc.)
# visualize_predictions(model, train_dataset, num_samples=3)


import torch
import segmentation_models_pytorch as smp

# --- CONFIG ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "best_rugged_ultra.pth"
ENCODER = 'efficientnet-b6'
CLASSES = 1

def load_rugged_model(path):
    # 1. Architecture initialize karein (Weights ke bina)
    model = smp.UnetPlusPlus(
        encoder_name=ENCODER,
        encoder_weights=None, # Loading ke waqt pretrained ki zaroorat nahi
        in_channels=3,
        classes=CLASSES,
    ).to(DEVICE)

    try:
        # Check karein ki file mein kya hai
        checkpoint = torch.load(path, map_location=DEVICE)
        
        # Agar checkpoint ek 'state_dict' hai
        if isinstance(checkpoint, dict):
            # Agar 'model_state_dict' key ke andar weights hain (common in training scripts)
            if 'model_state_dict' in checkpoint:
                model.load_state_dict(checkpoint['model_state_dict'])
            else:
                # Direct state_dict load karein
                # strict=False lagane se "missing keys" ignore ho jayengi aur jo match karega wo load hoga
                model.load_state_dict(checkpoint, strict=False)
        else:
            # Agar poora model hi save kiya tha (torch.save(model))
            model = checkpoint
            
        model.eval()
        print("✅ Model successfully loaded with flexibility!")
        return model

    except Exception as e:
        print(f"❌ Loading Error: {e}")
        return None

# --- USE THIS IN YOUR MAIN SCRIPT ---
# model = load_rugged_model(MODEL_PATH)
# if model:
#     visualize_predictions(model, your_dataset)


import torch
import segmentation_models_pytorch as smp

# --- CONFIG ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "best_rugged_ultra.pth"
ENCODER = 'efficientnet-b6'
CLASSES = 1

def load_rugged_model(path):
    # 1. Architecture initialize karein (Weights ke bina)
    model = smp.UnetPlusPlus(
        encoder_name=ENCODER,
        encoder_weights=None, # Loading ke waqt pretrained ki zaroorat nahi
        in_channels=3,
        classes=CLASSES,
    ).to(DEVICE)

    try:
        # Check karein ki file mein kya hai
        checkpoint = torch.load(path, map_location=DEVICE)
        
        # Agar checkpoint ek 'state_dict' hai
        if isinstance(checkpoint, dict):
            # Agar 'model_state_dict' key ke andar weights hain (common in training scripts)
            if 'model_state_dict' in checkpoint:
                model.load_state_dict(checkpoint['model_state_dict'])
            else:
                # Direct state_dict load karein
                # strict=False lagane se "missing keys" ignore ho jayengi aur jo match karega wo load hoga
                model.load_state_dict(checkpoint, strict=False)
        else:
            # Agar poora model hi save kiya tha (torch.save(model))
            model = checkpoint
            
        model.eval()
        print("✅ Model successfully loaded with flexibility!")
        return model

    except Exception as e:
        print(f"❌ Loading Error: {e}")
        return None

# --- USE THIS IN YOUR MAIN SCRIPT ---
# model = load_rugged_model(MODEL_PATH)
# if model:
#     visualize_predictions(model, your_dataset)
